## STEP 1: INSTALLATION & IMPORTS

In [ ]:
# --- 1. Installation ---
# Cài đặt tất cả các thư viện cần thiết.
# -qU: quiet (im lặng), upgrade (nâng cấp)
print("Bắt đầu cài đặt các thư viện...")
!pip install -qU langgraph langchain langchain-core langchain-community groq python-dotenv rich

Bắt đầu cài đặt các thư viện...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.5/152.5 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 63.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.4/131.4 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.4/243.4 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 13.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
b

In [ ]:
!pip install -qU tavily-python google-api-python-client google-auth-oauthlib
print("Đã cài đặt các thư viện công cụ cần thiết.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 70.6 MB/s eta 0:00:00
Đã cài đặt các thư viện công cụ cần thiết.


In [ ]:
!pip install google-auth-httplib2 langchain-groq

In [1]:
# --- 2. Core Imports ---
# Import các thư viện và lớp (class) cốt lõi sẽ được sử dụng trong toàn bộ dự án.
import os
import json
from typing import TypedDict, Annotated, List, Dict, Any
import operator
from google.colab import userdata

# LangChain & LangGraph core components
from langchain_core.messages import BaseMessage, HumanMessage, SystemMessage
from langgraph.graph import StateGraph, END, START

# Groq client for LLM interaction
from groq import AsyncGroq

# Rich library for beautiful printing
from rich.console import Console
from rich.markdown import Markdown

SHOW_DETAILED_LOGS = True

print("\nCài đặt và Import hoàn tất. Môi trường đã sẵn sàng!")


Cài đặt và Import hoàn tất. Môi trường đã sẵn sàng!


## STEP 2: STATE DEFINITION & LLM WRAPPER

In [2]:
# ==============================================================================
# STEP 2: STATE DEFINITION & LANGCHAIN LLM
# ==============================================================================
from typing import TypedDict, Annotated, List, Dict, Any, Literal, Optional
import operator
import json
import os
import tavily
from datetime import datetime, timezone, timedelta

from langchain_core.messages import BaseMessage, HumanMessage, SystemMessage, AIMessage, ToolMessage
from rich.console import Console
from rich.markdown import Markdown
from google.colab import userdata
from langchain_groq import ChatGroq
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from langchain_core.tools import tool
from langgraph.graph import StateGraph, END

# --- 1. State Definition (Hoàn chỉnh) ---
def merge_dicts(dict1: Dict[str, Any], dict2: Dict[str, Any]) -> Dict[str, Any]:
    merged = dict1.copy()
    for key, value in dict2.items():
        if key in merged and isinstance(merged[key], dict) and isinstance(value, dict):
            merged[key] = merge_dicts(merged[key], value)
        else: merged[key] = value
    return merged

class AcademicState(TypedDict):
    messages: Annotated[List[BaseMessage], operator.add]
    profile: Dict
    cross_agent_request: Optional[Dict[str, Any]]
    required_agents: List[str]
    completed_agents: Annotated[List[str], operator.add]
    agent_outputs: Annotated[Dict[str, Any], merge_dicts]
    critique: str
    revision_needed: bool
    revision_count: int
    final_report: str
    next_steps: str

print("Đã định nghĩa 'AcademicState' phiên bản cuối cùng thành công.")

# --- 2. Khởi tạo LLM ---
try:
    GROQ_API_KEY = userdata.get('GROQ_API_KEY')
    TAVILY_API_KEY = userdata.get('TAVILY_API_KEY')
    llm = ChatGroq(
        temperature=0.2, # Giảm độ "sáng tạo" để agent tuân thủ hướng dẫn hơn
        model="llama3-70b-8192",
        api_key=GROQ_API_KEY,
        max_tokens=4096
    )
    print(f"Khởi tạo ChatGroq thành công với model: {llm.model_name}")
except Exception as e:
    print(f"LỖI: Không thể khởi tạo LLM. {e}")
    llm = None
console = Console()
if llm: print("\nKhởi tạo 'llm' và 'console' thành công.")


# ==============================================================================
# STEP 3: THE TOOLBOX (PHIÊN BẢN SỬA LỖI CUỐI CÙNG)
# ==============================================================================
from google.oauth2.service_account import Credentials
from googleapiclient.discovery import build

# --- 1. Khởi tạo các service và dữ liệu ---
SERVICE_ACCOUNT_FILE = 'credentials.json'
# <-- SỬA ĐỔI QUAN TRỌNG: Thêm ID Lịch của bạn vào đây
YOUR_CALENDAR_ID = "td.tan2711@gmail.com" # <--- THAY BẰNG EMAIL CỦA BẠN

def authenticate_google_calendar_service_account():
    try:
        if not os.path.exists(SERVICE_ACCOUNT_FILE): return None
        SCOPES = ['https://www.googleapis.com/auth/calendar.readonly']
        creds = Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)
        service = build('calendar', 'v3', credentials=creds)
        print("✅ Xác thực Google Calendar thành công.")
        return service
    except Exception as e:
        print(f"Lỗi khi xác thực Google Calendar: {e}")
        return None
google_calendar_service = authenticate_google_calendar_service_account()

try:
    with open('tasks.json', 'r', encoding='utf-8') as f: task_data = json.load(f)
    print("✅ Đã tải dữ liệu tasks.json thành công.")
except Exception as e:
    task_data = {}
    print(f"Lỗi khi tải tasks.json: {e}")

try:
    with open('profile.json', 'r', encoding='utf-8') as f: profile_data = json.load(f)
    print("✅ Đã tải dữ liệu profile.json thành công.")
except Exception as e:
    profile_data = {}
    print(f"Lỗi khi tải profile.json: {e}")

# --- 2. Định nghĩa các công cụ ---
@tool
def google_calendar_reader(days_ahead: int = 7) -> str:
    """Sử dụng công cụ này để đọc lịch và các sự kiện sắp tới của người dùng."""
    if not google_calendar_service: return "Lỗi: Dịch vụ Google Calendar không có sẵn."
    now_utc = datetime.now(timezone.utc).isoformat()
    future_limit = (datetime.now(timezone.utc) + timedelta(days=days_ahead)).isoformat()
    try:
        print(f"--- [Calendar Tool] Đang đọc lịch: {YOUR_CALENDAR_ID}")
        # <-- SỬA ĐỔI QUAN TRỌNG: Sử dụng ID lịch của bạn thay vì 'primary'
        events_result = google_calendar_service.events().list(
            calendarId=YOUR_CALENDAR_ID,
            timeMin=now_utc,
            timeMax=future_limit,
            singleEvents=True,
            orderBy='startTime'
        ).execute()
        events = events_result.get('items', [])
        if not events:
            return f"Không có sự kiện nào sắp tới trong {days_ahead} ngày tới."
        print(f"--- [Calendar Tool] Tìm thấy {len(events)} sự kiện.")
        # Trả về một chuỗi JSON sạch
        return json.dumps([{'summary': e.get('summary'), 'start': e.get('start', {}).get('dateTime'), 'end': e.get('end', {}).get('dateTime')} for e in events])
    except Exception as e: return f"Lỗi khi gọi Google Calendar API: {e}"

@tool
def active_tasks_reader() -> str:
    """Sử dụng công cụ này để đọc danh sách các công việc và bài tập còn hoạt động của người dùng."""
    if not task_data: return "Không có dữ liệu công việc."
    now_utc = datetime.now(timezone.utc)
    active_tasks = [t for t in task_data.get("tasks", []) if t.get("status") == "needsAction" and t.get("due") and datetime.fromisoformat(t['due'].replace('Z', '+00:00')) > now_utc]
    return "Không có công việc nào còn hoạt động." if not active_tasks else json.dumps(active_tasks)

@tool
def tavily_web_search(query: str) -> str:
    """Sử dụng công cụ này để tìm kiếm thông tin học thuật trên Internet khi người dùng yêu cầu tìm hiểu về một chủ đề cụ thể."""
    try:
        client = tavily.TavilyClient(api_key=TAVILY_API_KEY)
        response = client.search(query=query, search_depth="advanced", max_results=5)
        return "\n\n".join([f'Source: {res["url"]}\nContent: {res["content"]}' for res in response["results"]])
    except Exception as e: return f"Lỗi khi tìm kiếm với Tavily: {e}"

# --- 3. Tạo danh sách và bind các công cụ ---
tools = [google_calendar_reader, active_tasks_reader, tavily_web_search]
llm_with_tools = llm.bind_tools(tools)
print("\n✅ Hộp dụng cụ (Toolbox) đã sẵn sàng và được bind vào LLM.")

/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py:3553: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


Đã định nghĩa 'AcademicState' phiên bản cuối cùng thành công.
Khởi tạo ChatGroq thành công với model: llama3-70b-8192

Khởi tạo 'llm' và 'console' thành công.
✅ Xác thực Google Calendar thành công.
✅ Đã tải dữ liệu tasks.json thành công.
✅ Đã tải dữ liệu profile.json thành công.

✅ Hộp dụng cụ (Toolbox) đã sẵn sàng và được bind vào LLM.


In [7]:
# ==============================================================================
# STEP 4: THE FULL INTERACTIVE TEAM OF AGENTS (KIẾN TRÚC CUỐI CÙNG)
# ==============================================================================
# --- 1. Định nghĩa các cấu trúc dữ liệu cho tương tác ---
class FinalAnswer(BaseModel):
    answer: str = Field(description="Nội dung câu trả lời cuối cùng, chi tiết bằng Markdown.")

class RequestForHelp(BaseModel):
    recipient: Literal["TOOL_USER"] = Field(description="Agent mà bạn muốn yêu cầu trợ giúp.")
    request: str = Field(description="Mô tả chi tiết yêu cầu của bạn.")

class Critique(BaseModel):
    critique: str = Field(description="Nhận xét chi tiết về sản phẩm.")
    revision_needed: bool = Field(description="True nếu sản phẩm cần làm lại, False nếu đã đạt chất lượng.")

# --- 2. Xây dựng toàn bộ đội ngũ Agent ---

async def coordinator_agent(state: AcademicState) -> Dict[str, Any]:
    print("\n--- [Coordinator] Đang quyết định các chuyên gia cần thiết... ---")
    prompt = f"Dựa trên yêu cầu: '{state['messages'][0].content}', quyết định các agent cần kích hoạt: PLANNER, NOTEWRITER, ADVISOR. Trả về một danh sách Python."
    response_message = await llm.ainvoke([HumanMessage(content=prompt)])
    try: required_agents = eval(response_message.content.strip())
    except: required_agents = ["PLANNER", "NOTEWRITER", "ADVISOR"]
    print(f"--- [Coordinator] Quyết định: Cần các agent {required_agents}")
    return {"required_agents": required_agents}

async def tool_user_agent(state: AcademicState) -> Dict[str, Any]:
    print("\n--- [Tool User] Đang làm việc... ---")
    output = {}
    cross_request = state.get("cross_agent_request")
    if cross_request:
        print(f"--- [Tool User] Nhận được yêu cầu trợ giúp từ {cross_request.get('sender')}: '{cross_request.get('request')}'")
        tool_user_messages = [state['messages'][0], HumanMessage(content=f"Yêu cầu trợ giúp: '{cross_request['request']}'. Hãy chọn công cụ phù hợp.")]
        output["cross_agent_request"] = None
    else:
        tool_user_messages = state["messages"]
    ai_response = await llm_with_tools.ainvoke(tool_user_messages)
    if not ai_response.tool_calls:
        return {"messages": [ai_response]}
    print(f"--- [Tool User] Quyết định gọi công cụ: {[tc['name'] for tc in ai_response.tool_calls]}")
    tool_outputs = []
    for tool_call in ai_response.tool_calls:
        time.sleep(1)
        tool_to_call = {t.name: t for t in tools}[tool_call["name"]]
        output_data = tool_to_call.invoke(tool_call["args"])
        tool_outputs.append(ToolMessage(content=str(output_data), tool_call_id=tool_call["id"]))
    output["messages"] = [ai_response] + tool_outputs
    return output

# Planner không còn chế độ sửa đổi nữa, nó luôn tạo mới
def create_specialist_agent(agent_name: str, agent_description: str, output_key: str):
    async def specialist_agent(state: AcademicState) -> Dict[str, Any]:
        print(f"\n--- [{agent_name}] Bắt đầu làm việc... ---")
        llm_with_specialist_tools = llm.bind_tools([FinalAnswer, RequestForHelp])
        prompt = f"""Bạn là {agent_name}. {agent_description}
        **Thông tin có sẵn (Lịch trình, nhiệm vụ, hồ sơ, kết quả tìm kiếm nếu có):**
        ```json
        {json.dumps({
            "profile": state['profile'],
            "messages": [msg.model_dump() for msg in state['messages'] if not (isinstance(msg, AIMessage) and not msg.content)],
            "agent_outputs": state['agent_outputs'],
        }, indent=2, ensure_ascii=False)}
        ```
        **QUYẾT ĐỊNH:** Dựa vào thông tin trên, hoặc gọi `FinalAnswer` nếu đủ thông tin, hoặc gọi `RequestForHelp` để tìm kiếm thêm.
        """
        try:
            ai_response = await llm_with_specialist_tools.ainvoke([HumanMessage(content=prompt)])
            if not ai_response.tool_calls: raise ValueError("LLM không gọi công cụ nào.")
            tool_call = ai_response.tool_calls[0]
            if tool_call['name'] == 'FinalAnswer':
                print(f"--- [{agent_name}] Đã tạo sản phẩm thành công.")
                return {"agent_outputs": {output_key: tool_call['args']['answer']}}
            elif tool_call['name'] == 'RequestForHelp':
                rfh = tool_call['args']; rfh['sender'] = agent_name
                print(f"--- [{agent_name}] Yêu cầu trợ giúp: {rfh}")
                return {"cross_agent_request": rfh}
            else: raise ValueError(f"LLM đã gọi một công cụ không xác định: {tool_call['name']}")
        except Exception as e:
            print(f"--- [{agent_name}] Lỗi nghiêm trọng: {e}")
            return {"cross_agent_request": {'sender': agent_name, 'recipient': 'TOOL_USER', 'request': f'Đã xảy ra lỗi. Hãy tìm kiếm thông tin về chủ đề chính: "{state["messages"][0].content}"'}}
    return specialist_agent

planner_agent = create_specialist_agent("PLANNER", "Tạo kế hoạch học tập chi tiết, khả thi.", "plan")
notewriter_agent = create_specialist_agent("NOTEWRITER", "Tạo ghi chú học thuật súc tích.", "notes")
advisor_agent = create_specialist_agent("ADVISOR", "Đưa ra lời khuyên chiến lược.", "advice")

# Critic agent giờ đây, nếu cần sửa, sẽ tạo ra một yêu cầu trợ giúp cho Tool User
async def critic_agent(state: AcademicState) -> Dict[str, Any]:
    print("\n--- [Critic] Bắt đầu đánh giá... ---")
    output_to_review = state.get("agent_outputs", {}).get("plan")
    if not output_to_review:
        return {"revision_needed": True, "critique": "Lỗi: Không tìm thấy sản phẩm từ Planner."}

    llm_with_critique_tool = llm.bind_tools([Critique])
    prompt = f"Bạn là Critic Agent. Đánh giá kế hoạch học tập này:\n---\n{output_to_review}\n---\nTiêu chí: Kế hoạch có chi tiết và cung cấp tài nguyên cụ thể không? Gọi công cụ `Critique` với kết quả."
    try:
        ai_response = await llm_with_critique_tool.ainvoke([HumanMessage(content=prompt)])
        if not ai_response.tool_calls: raise ValueError("Critic không gọi công cụ Critique.")
        critique_args = ai_response.tool_calls[0]['args']
        print(f"--- [Critic] Kết quả đánh giá: {critique_args.get('critique')} | Cần làm lại: {critique_args.get('revision_needed')}")

        # <-- SỬA ĐỔI QUAN TRỌNG: Nếu cần sửa đổi, tạo một cross_agent_request
        if critique_args.get('revision_needed'):
            # Tạo một yêu cầu tìm kiếm thông minh dựa trên nhận xét
            search_request = f"Kế hoạch học tập về KNN còn chung chung. Hãy tìm kiếm các tài liệu, video, hoặc bài tập thực hành cụ thể về KNN để cải thiện kế hoạch. Gợi ý từ Critic: {critique_args.get('critique')}"
            return {
                "revision_needed": True,
                "cross_agent_request": {"sender": "CRITIC", "recipient": "TOOL_USER", "request": search_request}
            }
        else:
            return {"revision_needed": False, "critique": critique_args.get('critique')}
    except Exception as e:
        print(f"--- [Critic] Lỗi nghiêm trọng: {e}")
        return {"revision_needed": True, "critique": "Lỗi hệ thống trong Critic Agent."}

# Các agent cuối giữ nguyên
async def synthesizer_agent(state: AcademicState) -> Dict[str, Any]:
    print("\n--- [Synthesizer] Bắt đầu tổng hợp báo cáo cuối cùng ---")
    prompt = f"Tổng hợp các sản phẩm sau thành một báo cáo duy nhất: {json.dumps(state['agent_outputs'], indent=2, ensure_ascii=False)}."
    response_message = await llm.ainvoke([HumanMessage(content=prompt)])
    return {"final_report": response_message.content}

async def reflection_agent(state: AcademicState) -> Dict[str, Any]:
    print("\n--- [Reflection] Đang suy nghĩ về các bước tiếp theo... ---")
    prompt = f"Dựa vào báo cáo cuối cùng, đề xuất 3-4 hành động tiếp theo: {state['final_report']}"
    response_message = await llm.ainvoke([HumanMessage(content=prompt)])
    return {"next_steps": response_message.content}

print("✅ Tất cả các agent cho ATLAS 2.0 (Kiến trúc dứt điểm) đã được định nghĩa.")

✅ Tất cả các agent cho ATLAS 2.0 (Kiến trúc dứt điểm) đã được định nghĩa.


In [8]:
# ==============================================================================
# STEP 5 & 6: ASSEMBLY & EXECUTION (KIẾN TRÚC DỨT ĐIỂM)
# ==============================================================================
# --- Logic điều khiển (Routers) ---
def master_router(state: AcademicState) -> str:
    if state.get("cross_agent_request"):
        return "tool_user"
    required = state.get("required_agents", [])
    completed = state.get("completed_agents", [])
    next_agent = next((agent for agent in required if agent not in completed), None)
    if next_agent:
        return next_agent.lower()
    return "synthesizer"

# <-- SỬA ĐỔI QUAN TRỌNG: Router này giờ đây cực kỳ đơn giản
def revision_router(state: AcademicState) -> Literal["mark_planner_complete", "tool_user"]:
    print("\n--- [Revision Router] Đang kiểm tra kết quả phê bình...")
    # Nếu Critic yêu cầu giúp đỡ, đi đến Tool User
    if state.get("cross_agent_request"):
        print("--- [Revision Router] Critic yêu cầu thông tin bổ sung. Chuyển đến Tool User.")
        return "tool_user"
    else:
        # Nếu Critic không yêu cầu giúp đỡ, nghĩa là kế hoạch đã OK
        print("--- [Revision Router] Đạt chất lượng. Đánh dấu Planner hoàn thành.")
        return "mark_planner_complete"

# --- Các node hành động ---
async def mark_planner_complete(state: AcademicState) -> Dict[str, Any]:
    print("\n--- [State Manager] Đánh dấu PLANNER đã hoàn thành.")
    return {"completed_agents": ["PLANNER"]}

# Node tóm tắt bộ nhớ giữ nguyên
async def summarize_memory(state: AcademicState) -> Dict[str, Any]:
    print("\n--- [Memory Manager] Đang tóm tắt và dọn dẹp bộ nhớ...")
    last_agent = state.get("completed_agents", [])[-1]
    last_output = state.get("agent_outputs", {}).get(last_agent.lower())
    prompt = f"Tóm tắt kết quả: {last_agent} đã hoàn thành và tạo ra: {last_output}. Tóm tắt trong 1 câu."
    summary_message = await llm.ainvoke([HumanMessage(content=prompt)])
    new_messages = [state['messages'][0], SystemMessage(content=f"Bối cảnh: {summary_message.content}")]
    print(f"--- [Memory Manager] Tóm tắt mới: {summary_message.content}")
    return {"messages": new_messages}

# Node wrapper cho các agent đơn giản
async def simple_specialist_node(agent_func, agent_name: str, state: AcademicState) -> Dict[str, Any]:
    result = await agent_func(state)
    if "cross_agent_request" not in result:
        print(f"--- [Node Runner] Đánh dấu {agent_name} đã hoàn thành.")
        current_completed = state.get("completed_agents", [])
        if agent_name not in current_completed:
             state['completed_agents'] = current_completed + [agent_name]
    return result

notewriter_node = lambda st: simple_specialist_node(notewriter_agent, "NOTEWRITER", st)
advisor_node = lambda st: simple_specialist_node(advisor_agent, "ADVISOR", st)

# --- Lắp ráp Graph ---
workflow = StateGraph(AcademicState)
workflow.add_node("coordinator", coordinator_agent)
workflow.add_node("tool_user", tool_user_agent)
workflow.add_node("planner", planner_agent)
workflow.add_node("critic", critic_agent)
workflow.add_node("mark_planner_complete", mark_planner_complete)
workflow.add_node("notewriter", notewriter_node)
workflow.add_node("advisor", advisor_node)
workflow.add_node("summarize_memory", summarize_memory)
workflow.add_node("synthesizer", synthesizer_agent)
workflow.add_node("reflection", reflection_agent)

workflow.set_entry_point("coordinator")

workflow.add_edge("coordinator", "tool_user")
workflow.add_conditional_edges("tool_user", master_router, {
    "planner": "planner", "notewriter": "notewriter", "advisor": "advisor",
    "synthesizer": "synthesizer", "tool_user": "tool_user"
})

# Planner luôn đi đến Critic
workflow.add_edge("planner", "critic")

# <-- SỬA ĐỔI QUAN TRỌNG: Luồng phản biện mới
workflow.add_conditional_edges("critic", revision_router, {
    # Nếu cần sửa, Critic sẽ tạo yêu cầu cho Tool User, router sẽ định tuyến đến đó
    "tool_user": "tool_user",
    # Nếu OK, đi đánh dấu hoàn thành
    "mark_planner_complete": "mark_planner_complete"
})

# Sau khi các agent hoàn thành, đi tóm tắt bộ nhớ
workflow.add_edge("mark_planner_complete", "summarize_memory")
workflow.add_edge("notewriter", "summarize_memory")
workflow.add_edge("advisor", "summarize_memory")

workflow.add_edge("summarize_memory", "tool_user")

workflow.add_edge("synthesizer", "reflection")
workflow.add_edge("reflection", END)

app = workflow.compile()
print("\n✅✅✅ ATLAS 2.0 (Kiến trúc dứt điểm) đã được biên dịch! ✅✅✅")


✅✅✅ ATLAS 2.0 (Kiến trúc dứt điểm) đã được biên dịch! ✅✅✅


In [9]:
# ==============================================================================
# STEP 7: EXECUTION (Chạy hệ thống hoàn chỉnh)
# ==============================================================================

async def run_atlas_2_0(profile_data: dict):
    # Bạn có thể thay đổi yêu cầu ở đây để thử nghiệm
    request = "Tuần tới tôi cần học về thuật toán K-Nearest Neighbors (KNN). Lịch của tôi khá bận rộn, hãy tạo một kế hoạch học tập chi tiết, sau đó ghi chú lại các điểm chính và cho tôi một vài lời khuyên học tập."

    # Lấy hồ sơ sinh viên
    student_profile = next((p for p in profile_data.get("profiles", []) if p.get("id") == "student_123"), {})

    # Thiết lập trạng thái ban đầu cho graph
    initial_state = {
        "messages": [HumanMessage(content=request)],
        "profile": student_profile,
        "agent_outputs": {},
        "completed_agents": [],
        "revision_count": 0,
        "cross_agent_request": None,
    }

    console.print(f"\n🚀 [bold blue]Bắt đầu chạy ATLAS 2.0 với yêu cầu:[/bold blue]\n{request}")

    final_state_snapshot = None
    # <-- SỬA ĐỔI QUAN TRỌNG: Thêm config vào đây
    async for step in app.astream(initial_state, config={"recursion_limit": 50}):
        node_name = list(step.keys())[0]
        console.print(f"\n✅ [bold magenta]Hoàn thành bước:[/] [yellow]{node_name}[/yellow]")
        # In ra nội dung của state sau mỗi bước để gỡ lỗi và theo dõi
        console.print(step[node_name])
        final_state_snapshot = step[node_name]

    # In kết quả cuối cùng
    console.print("\n\n" + "="*80, style="bold green")
    console.print(" HỆ THỐNG ATLAS 2.0 ĐÃ HOÀN THÀNH TÁC VỤ!", style="bold green")
    console.print("="*80, style="bold green")


    if final_state_snapshot:
        console.print("\n--- [bold blue]BÁO CÁO TỔNG HỢP CUỐI CÙNG[/bold blue] ---")
        console.print(Markdown(final_state_snapshot.get("final_report", "Không có báo cáo cuối cùng.")))
        console.print("\n--- [bold blue]GỢI Ý CÁC BƯỚC TIẾP THEO[/bold blue] ---")
        console.print(Markdown(final_state_snapshot.get("next_steps", "Không có gợi ý.")))

# Chạy hàm chính
await run_atlas_2_0(profile_data)

🚀 Bắt đầu chạy ATLAS 2.0 với yêu cầu:
Tuần tới tôi cần học về thuật toán K-Nearest Neighbors (KNN). Lịch của tôi khá bận rộn, hãy tạo một kế hoạch học 
tập chi tiết, sau đó ghi chú lại các điểm chính và cho tôi một vài lời khuyên học tập.


--- [Coordinator] Đang quyết định các chuyên gia cần thiết... ---
--- [Coordinator] Quyết định: Cần các agent ['PLANNER', 'NOTEWRITER', 'ADVISOR']


✅ Hoàn thành bước: coordinator

{'required_agents': ['PLANNER', 'NOTEWRITER', 'ADVISOR']}


--- [Tool User] Đang làm việc... ---
--- [Tool User] Quyết định gọi công cụ: ['google_calendar_reader']
--- [Calendar Tool] Đang đọc lịch: td.tan2711@gmail.com
--- [Calendar Tool] Tìm thấy 4 sự kiện.


...

{
    'messages': [
        HumanMessage(
            content='Tuần tới tôi cần học về thuật toán K-Nearest Neighbors (KNN). Lịch của tôi khá bận rộn, hãy 
tạo một kế hoạch học tập chi tiết, sau đó ghi chú lại các điểm chính và cho tôi một vài lời khuyên học tập.',
            additional_kwargs={},
            response_metadata={}
        ),
        SystemMessage(
            content='Bối cảnh: PLANNER đã hoàn thành nhưng không tạo ra kết quả nào.',
            additional_kwargs={},
            response_metadata={}
        )
    ]
}


--- [Tool User] Đang làm việc... ---


✅ Hoàn thành bước: tool_user

{
    'messages': [
        AIMessage(
            content="Based on your schedule, I've created a detailed learning plan for you to learn about the 
K-Nearest Neighbors (KNN) algorithm. Here's a suggested plan:\n\n**Day 1: Introduction to KNN (30 minutes)**\n\n* 
Read the introduction to KNN from the provided resources (e.g., DataCamp, IBM, Analytics Vidhya, GeeksforGeeks)\n* 
Understand the basic concept of KNN, its applications, and importance\n* Take notes on the key points\n\n**Day 2: 
Understanding KNN Algorithm (60 minutes)**\n\n* Read about the KNN algorithm in detail (e.g., how it works, steps 
involved, advantages, and disadvantages)\n* Watch video tutorials or online lectures to supplement your 
understanding\n* Take notes on the key points and summarize the algorithm in your own words\n\n**Day 3: 
Implementing KNN in Python (90 minutes)**\n\n* Choose a Python library (e.g., scikit-learn) and implement a simple 
KNN algorithm\n* Practice coding exercises or work on a project to solidify your understanding\n* Take notes on any
challenges you face and how you overcome them\n\n**Day 4: KNN Applications and Case Studies (60 minutes)**\n\n* 
Read about real-world applications of KNN (e.g., image classification, recommender systems, anomaly detection)\n* 
Study case studies or examples of KNN in action\n* Take notes on how KNN is used in different domains and 
industries\n\n**Day 5: Review and Practice (90 minutes)**\n\n* Review all the concepts learned throughout the 
week\n* Practice coding exercises or work on a project to reinforce your understanding\n* Take notes on any areas 
where you need more practice or review\n\n**Additional Tips:**\n\n* Set aside dedicated time each day to focus on 
learning KNN\n* Take breaks to avoid burnout and retain information better\n* Practice coding exercises or work on 
projects to solidify your understanding\n* Join online communities or forums to ask questions and get feedback\n* 
Review and practice regularly to reinforce your learning\n\nBy following this plan, you'll have a solid 
understanding of the KNN algorithm and its applications. Remember to practice regularly and review the concepts to 
reinforce your learning. Good luck!",
            additional_kwargs={},
            response_metadata={
                'token_usage': {
                    'completion_tokens': 459,
                    'prompt_tokens': 4594,
                    'total_tokens': 5053,
                    'completion_time': 2.169292775,
                    'prompt_time': 0.179073475,
                    'queue_time': 0.012587043,
                    'total_time': 2.3483662499999998
                },
                'model_name': 'llama3-70b-8192',
                'system_fingerprint': 'fp_bf16903a67',
                'service_tier': 'on_demand',
                'finish_reason': 'stop',
                'logprobs': None
            },
            id='run--fd51786d-6349-4ffa-ba3f-af71c4b5dae0-0',
            usage_metadata={'input_tokens': 4594, 'output_tokens': 459, 'total_tokens': 5053}
        )
    ]
}

InvalidUpdateError: Expected dict, got <coroutine object simple_specialist_node at 0x7b1c0e914f40>
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_GRAPH_NODE_RETURN_VALUE